In [6]:
% pylab inline

Populating the interactive namespace from numpy and matplotlib


# Compute correction function

### First estimate pair counts


In [68]:
n=400
m=100
RR_file = '/mnt/alan1/oliverphilcox/PowerSpectra/RR_counts_n%d_m%d_11.txt'%(n,m)
RR_flat = np.loadtxt(RR_file)#*np.sum(gal_w)**2.# must remove normalization factor
RR_true = RR_flat.reshape((n,m))[:,:]

## Define bins
mu_cen = np.arange(1/(2*m),1+1/(2*m),1/m)
r_bins = np.loadtxt('/mnt/store1/oliverphilcox/PowerSpectra/radial_binning_fine.csv')
r_cen = mean(r_bins,axis=1)
delta_r = r_cen[-1]-r_cen[-2]
mu_cen = np.arange(1/(2*m),1.+1/(2*m),1/m)
delta_mu = mu_cen[-1]-mu_cen[-2]
assert n==len(r_cen)
assert m==len(mu_cen)

In [77]:
r_max = 400
nrbins=100
r_bins_all = np.linspace(0,r_max,nrbins+1)
r_bins_low = r_bins_all[:-1]
r_bins_hi = r_bins_all[1:]
binfile = np.vstack([r_bins_low,r_bins_hi])


In [79]:
from Corrfunc.theory import DDsmu

In [70]:
def RR_model(r_cen,mu):
    return norm*((r_cen+delta_r/2.)**3.-(r_cen-delta_r/2.)**3.)*delta_mu/3.
    #return norm*r_cen**2.*delta_r*delta_mu
    
mumu,rr = np.meshgrid(mu_cen,r_cen)
Phi_true = RR_model(rr,mumu)/RR_true

inv_phi = RR_true/RR_model(rr,mumu)

l_max = 8
from scipy.special import legendre
inv_Phi_multipoles = np.zeros([l_max//2+1,len(inv_phi)])
for i in range(len(RR_true)):
    for l_i,ell in enumerate(np.arange(0,l_max+2,2)):
        inv_Phi_multipoles[l_i,i]=(2.*ell+1.)*delta_mu*np.sum(legendre(ell)(mu_cen)*inv_phi[i,:])
        
        
def inv_phi_ell_model(r,*par):
    return par[0]+par[1]*r+par[2]*r**2.#+par[3]*r**3.#+par[4]*r**4.
from scipy.optimize import curve_fit
max_l = 4
all_ell = np.arange(0,max_l+2,2)
coeff = np.asarray([curve_fit(inv_phi_ell_model,r_cen[1:],inv_Phi_multipoles[ell//2][1:],
                              p0=[0 for _ in range(3)])[0] for ell in all_ell])

np.savetxt("/mnt/store1/oliverphilcox/PowerSpectra/InvPhiCoeff_DR12.txt",coeff,delimiter="\t")

In [71]:
coeff

array([[ 1.19291832e+01, -1.24744518e-02, -2.23309629e-06],
       [ 1.73524678e-01, -8.35790320e-03, -5.00198798e-06],
       [-8.93255294e-04,  2.61453168e-03, -4.21120663e-07]])

# Reconstruct Power Spectrum

In [60]:
import numpy as np
import sys

DD_input = '/mnt/alan1/oliverphilcox/PowerQPM_New/R0_100/DD_mock_0001_power_counts_n19_m3_full.txt'
DR_input = '/mnt/alan1/oliverphilcox/PowerQPM_New/R0_100/DR_mock_0001_power_counts_n19_m3_full.txt'
RR_input = '/mnt/alan1/oliverphilcox/PowerQPM_New/R0_100/RR_0_power_counts_n19_m3_full.txt'
gal_file = '/mnt/alan1/oliverphilcox/QPM_proc/qpm_galaxy_0001.xyzw'
N_gal = 642051
N_rand_RR = 32292068
N_rand_DR = 642051
periodic = False
outfile = './test_power.txt'

In [62]:
# Load pair counts
DD = np.loadtxt(DD_input)
DR = np.loadtxt(DR_input)
RR = np.loadtxt(RR_input)
DR_pairs = N_gal*N_rand_DR
DD_pairs = N_gal*(N_gal-1)
RR_pairs = N_rand_RR*(N_rand_RR-1.)

# Compute normalization factor
print('Computing normalization factor')
if periodic:
    norm = 0.
    gal_x,gal_y,gal_z,gal_w = np.loadtxt(gal_file).T
    x_range = max(gal_x)-min(gal_x)
    y_range = max(gal_y)-min(gal_y)
    z_range = max(gal_z)-min(gal_z)
    
    if np.abs(x_range/y_range-1.)>0.01 or np.abs(x_range/z_range-1.)>0.01:
        print('Is this data periodic? Different x, y and z axis dimensions found. Exiting')
        sys.exit();
        
    V = x_range*y_range*z_range
    n_gal = N_gal/V
    norm = np.mean(gal_w**2.)*n_gal**2.*V
    print('Norm = %.2f'%norm)
else:
    norm = 0
    with open(gal_par) as infile:
        for l,line in enumerate(infile):
            this_w = float(line.split()[3])
            # Assume FKP weights here
            norm+=this_w**2.*(1./this_w-1.)/20000
    print("Norm = %.2f"%norm)
    
## Compute power

power = (DD-2.*DR/(DR_pairs/DD_pairs)+RR/(RR_pairs/DD_pairs))/norm

# Save to file
np.savetxt(outfile,power)
print('Saved power spectrum estimate to %s'%outfile)

Computing normalization factor
Norm = 4.25
Saved power spectrum estimate to ./test_power.txt


In [59]:
### compute power
power = (DD-2.*DR/(DR_pairs/DD_pairs)+RR/(RR_pairs/DD_pairs))/norm